In [127]:
import os
from dotenv import load_dotenv

import pandas as pd

from langchain_community.document_loaders import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

from pinecone import Pinecone, ServerlessSpec

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

CODE TO EXTRACT URLS FROM MODEL'S OUTPUT


In [128]:
import re

def extract_urls_list(text: str):
    """
    Extract all URLs from the model's text output.
    Returns a list of cleaned, unique URLs (ordered).
    """
    if not isinstance(text, str):
        return []

    urls = re.findall(r'(https?://\S+)', text)
    # Clean trailing punctuation
    cleaned_urls = [u.rstrip('.,);') for u in urls]

    # Remove duplicates but keep order
    seen = set()
    unique_urls = []
    for u in cleaned_urls:
        if u not in seen:
            seen.add(u)
            unique_urls.append(u)

    return unique_urls


LOADING ENVIRONMENT VARIABLES 

In [129]:
load_dotenv()  # Loads PINECONE_API_KEY, GROQ_API_KEY, etc. from .env

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_ACCESS_TOKEN")
INDEX_NAME = "shl-individual-tests"
PINECONE_CLOUD = "aws"
PINECONE_REGION = "us-east-1"
CSV_PATH = "shl_individual_tests.csv"

DOCUMENT INGESTION

In [130]:
#DATA LOADING
def load_shl_catalog(csv_path: str):
    """
    Load the SHL individual tests CSV as Documents using CSVLoader.
    Each row becomes a Document with all columns in the page_content.
    """
    loader = CSVLoader(file_path=csv_path, encoding="utf-8")
    documents = loader.load()
    print(f"Loaded {len(documents)} documents from {csv_path}")
    return documents

#GENERATING EMBEDDINGS

In [131]:
#Generating Embeddings

def get_embeddings():
    """
    HuggingFace sentence-transformer embeddings.
    You can change model_name if needed.
    """
    model_name = "sentence-transformers/all-mpnet-base-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

VECTOR DB STORAGE

In [132]:
# Vector DB STORAGE
def init_pinecone_and_vectorstore(documents, embeddings):
    """
    Initializes Pinecone client, creates index (if needed),
    and builds a LangChain PineconeVectorStore from documents.

    IMPORTANT:
    - Only upsert documents when creating the index.
    - When index already exists, just connect using from_existing_index.
    """
    pc = Pinecone(api_key=PINECONE_API_KEY)

    # Check if index already exists
    existing_indexes = [idx["name"] for idx in pc.list_indexes()]
    
    if INDEX_NAME not in existing_indexes:
        # all-mpnet-base-v2 has 768-dim embeddings
        pc.create_index(
            name=INDEX_NAME,
            dimension=768,
            metric="cosine",
            spec=ServerlessSpec(
                cloud=PINECONE_CLOUD,
                region=PINECONE_REGION,
            ),
        )
        print(f"Created Pinecone index: {INDEX_NAME}")

        # First time: create and upsert all documents
        vectorstore = PineconeVectorStore.from_documents(
            documents=documents,
            embedding=embeddings,
            index_name=INDEX_NAME,
        )
    else:
        print(f"Using existing Pinecone index: {INDEX_NAME}")

        # Subsequent calls: just connect to existing index, DO NOT upsert
        vectorstore = PineconeVectorStore.from_existing_index(
            index_name=INDEX_NAME,
            embedding=embeddings,
        )

    return vectorstore


LLM CALLING

In [134]:
def get_llm():
    """
    Returns a ChatGroq LLM instance.
    Change model name if you prefer another Groq-hosted model.
    """
    llm = ChatGroq(
        groq_api_key=GROQ_API_KEY,
        model_name="llama-3.1-8b-instant",  # or another Groq model
        temperature=0.2,
        max_tokens=512,
    )
    return llm

RETRIEVAL CHAIN

In [136]:
# RAG PROMPT CHAIN
def build_rag_chain(vectorstore, llm):
    """
    Build a RAG chain that:
      - Retrieves diverse top-k catalog entries from Pinecone
      - Passes them + user question into an LLM
      - Asks LLM to pick and recommend minimum 5 DISTINCT test solutions ,maximum limit is 10
    """

    # Retriever: use MMR to increase diversity and fetch more candidates
    retriever = vectorstore.as_retriever(
        search_type="mmr",                     
        search_kwargs={
            "k": 30,                           
            "fetch_k": 60,                     
            "lambda_mult": 0.8,                
        },
    )

    prompt = ChatPromptTemplate.from_template(
        """
You are an expert SHL test catalog assistant.
You are given:
- A user requirement (what kind of candidate or skill they want to assess).
- A catalog context containing multiple individual tests.
Each test in the catalog has at least:
- An assessment/test name
- A URL (assessment URL)

Your job:
1. Read the following catalog entries from the SHL individual tests dataset.
2. Based on the user's requirement, choose atleast 5 MOST relevant individual test solutions, maximum solutions is 10.
3. VERY IMPORTANT: 
   - All  recommended tests MUST be different.
   - Each recommendation MUST have a DIFFERENT URL.
   - Never repeat the same URL more than once.
   - If the same URL appears many times in the context, treat it as ONE test and only recommend it once.
4. Prefer tests that match:
   - The technology / role / skill mentioned in the query
   - The seniority level (junior / mid / senior) if provided
   - Any test-specific needs (e.g., finance, sales, leadership, data entry, data science)
5. If there are not enough distinct URLs in the context to recommend 5 completely different tests,
   then recommend as many as possible and explicitly say that fewer than 5 unique URLs were found.

Important rules:
- ONLY recommend tests that appear in the catalog context below.
- If multiple tests look similar, prioritize the ones that are broader and more standard.
- Do NOT invent new tests or URLs that are not in the context.
- Do NOT show the same test or URL more than once in the final answer.

<catalog_context>
{context}
</catalog_context>

User requirement:
{question}

Return the final answer in this format (exactly):

Five Recommended Individual Test Solutions:
1. <Test Name 1>
   URL: <url1>


If you can only find fewer than 5 DISTINCT URLs, still use the same numbered format
but fill as many as you can and add a short note explaining why there are fewer than 5.
        """.strip()
    )

    # RAG pipeline: {context <- retriever, question <- input} -> prompt -> llm -> text
    rag_chain = (
        RunnableParallel(
            context=retriever,
            question=RunnablePassthrough()
        )
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain


REFINING THE SEARCH

In [137]:
#Recommend 5 solutions
def recommend_individual_tests(user_requirement: str) -> str:
    """
    High-level function:
      - Loads data (if needed)
      - Builds embeddings, Pinecone vectorstore, LLM
      - Runs RAG chain to recommend minimum 5 most relevant individual test solutions,maximum number of test solutions is 10.
    Returns the final text answer from the LLM.
    """

    # 1) Load catalog documents
    documents = load_shl_catalog(CSV_PATH)

    # 2) Embeddings
    embeddings = get_embeddings()

    # 3) Vector store (Pinecone)
    vectorstore = init_pinecone_and_vectorstore(documents, embeddings)

    # 4) LLM
    llm = get_llm()

    # 5) RAG chain
    rag_chain = build_rag_chain(vectorstore, llm)

    # 6) Invoke chain with the user requirement
    answer = rag_chain.invoke(user_requirement)
    return answer


RUNNING CODE ON TRAIN SET

In [138]:
if __name__ == "__main__":
    import pandas as pd

    excel_path = "Gen_AI Dataset.xlsx"   # your Excel file
    sheet_name = "Train-Set"                 # or "Train-Set" – match your sheet
    query_col = "Query"                  # column that has the queries

    # Load sheet
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    # Get unique, non-null queries so each is processed once
    unique_queries = df[query_col].dropna().drop_duplicates()

    print(f"Total rows in original sheet      : {len(df)}")
    print(f"Unique non-null queries processed : {len(unique_queries)}")

    # We'll build a new list of rows: one per (query, url)
    rows = []

    for i, query in enumerate(unique_queries, start=1):
        print("\n======================================")
        print(f"Processing unique query {i}/{len(unique_queries)}:")
        print(query)
        print("--------------------------------------")

        # Call your existing RAG function
        answer = recommend_individual_tests(query)

        # Extract ALL URLs from the model output
        urls = extract_urls_list(answer)

        if not urls:
            # Still store a row with empty URL if nothing found (optional)
            rows.append({
                query_col: query,
                "Model_URL": ""
            })
            print("No URLs found for this query.")
        else:
            print("URLs found:")
            for u in urls:
                print("  ", u)
                rows.append({
                    query_col: query,
                    "Model_URL": u
                })

    # Create a new DataFrame with one URL per row
    df_urls = pd.DataFrame(rows)

    # Save to a new Excel file
    output_path = "TrainData12.xlsx"
    df_urls.to_excel(output_path, sheet_name="Train-Set", index=False)

    print("\nFinished running all unique queries ✅")
    print(f"Results saved to: {output_path}")


Total rows in original sheet      : 65
Unique non-null queries processed : 10

Processing unique query 1/10:
I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.
--------------------------------------
Loaded 377 documents from shl_individual_tests.csv
Using existing Pinecone index: shl-individual-tests
URLs found:
   https://www.shl.com/products/product-catalog/view/java-2-platform-enterprise-edition-1-4-fundamental/
   https://www.shl.com/products/product-catalog/view/java-design-patterns-new/
   https://www.shl.com/products/product-catalog/view/core-java-entry-level-new/
   https://www.shl.com/products/product-catalog/view/java-frameworks-new/
   https://www.shl.com/products/product-catalog/view/smart-interview-live-coding/

Processing unique query 2/10:
I want to hire new graduates for a sales role in my company, the budget is for about an hour for each test. Give me some option

RUNNING CODE ON TEST SET

In [ ]:
#DOING THE SAME FOR THE TEST SET NOW
if __name__ == "__main__":
    import pandas as pd

    excel_path = "Gen_AI Dataset.xlsx"   # your Excel file
    sheet_name = "Test-Set"                 # or "Train-Set" 
    query_col = "Query"                  # column that has the queries

    # Load sheet
    df = pd.read_excel(excel_path, sheet_name=sheet_name)

    # Get unique, non-null queries so each is processed once
    unique_queries = df[query_col].dropna().drop_duplicates()

    print(f"Total rows in original sheet      : {len(df)}")
    print(f"Unique non-null queries processed : {len(unique_queries)}")

    # We'll build a new list of rows: one per (query, url)
    rows = []

    for i, query in enumerate(unique_queries, start=1):
        print("\n======================================")
        print(f"Processing unique query {i}/{len(unique_queries)}:")
        print(query)
        print("--------------------------------------")

        # Call your existing RAG function
        answer = recommend_individual_tests(query)

        # Extract ALL URLs from the model output
        urls = extract_urls_list(answer)

        if not urls:
            # Still store a row with empty URL if nothing found (optional)
            rows.append({
                query_col: query,
                "Model_URL": ""
            })
            print("No URLs found for this query.")
        else:
            print("URLs found:")
            for u in urls:
                print("  ", u)
                rows.append({
                    query_col: query,
                    "Model_URL": u
                })

    # Create a new DataFrame with one URL per row
    df_urls = pd.DataFrame(rows)

    # Save to a new Excel file
    output_path = "Testdata1.xlsx"
    df_urls.to_excel(output_path, sheet_name="Test-Set", index=False)

    print("\nFinished running all unique queries ✅")
    print(f"Results saved to: {output_path}")


Total rows in original sheet      : 9
Unique non-null queries processed : 9

Processing unique query 1/9:
Looking to hire mid-level professionals who are proficient in Python, SQL and Java Script. Need an assessment package that can test all skills with max duration of 60 minutes.
--------------------------------------
Loaded 377 documents from shl_individual_tests.csv
Using existing Pinecone index: shl-individual-tests
URLs found:
   https://www.shl.com/products/product-catalog/view/global-skills-assessment/
   https://www.shl.com/products/product-catalog/view/smart-interview-on-demand/
   https://www.shl.com/products/product-catalog/view/entry-level-technical-support-solution/
   https://www.shl.com/products/product-catalog/view/agile-testing-new/
   https://www.shl.com/products/product-catalog/view/workplace-administration-skills-new/

Processing unique query 2/9:
Job Description

 Join a community that is shaping the future of work! 

 SHL, People Science. People Answers. 

Are you

In [ ]:
import pandas as pd
import numpy as np
# Helper: Recall@K for a single query
def recall_at_k(retrieved_urls, relevant_urls, k: int) -> float:
    """
    retrieved_urls: list of URLs returned by the model (ranked).
    relevant_urls: list of ALL ground-truth URLs for this query.
    """
    if not relevant_urls:
        return 0.0

    retrieved_top_k = retrieved_urls[:k]
    relevant_set = set(relevant_urls)

    retrieved_relevant = sum(url in relevant_set for url in retrieved_top_k)
    return retrieved_relevant / len(relevant_set)


# === Load data ===

#  (train-set with relevant URLs, 1 row per URL)
train_df = pd.read_excel("Gen_AI Dataset.xlsx", sheet_name="Train-Set")

# Predictions (1 row per predicted URL)
pred_df = pd.read_excel("TrainData1.xlsx", sheet_name="Train-Set")

query_col    = "Query"
pred_col     = "Model_URL"

# 🔥 Change this to the actual column name in your train-set
# which holds the relevant (ground truth) URLs, 1 per row.
relevant_col = "Assessment_url"   # e.g. "Relevant_URL" or "Correct_URL"

K = 10  # Recall@K


# === Compute Recall@K per query ===

recall_scores = []
eval_rows = []

# Use all queries that appear in the ground-truth train_df
for query in train_df[query_col].dropna().drop_duplicates():

    # All predicted URLs for this query (could be more than K; we'll cut to K inside recall_at_k)
    predicted_urls = (
        pred_df.loc[pred_df[query_col] == query, pred_col]
        .tolist()
    )

    # All relevant URLs for this query (since they are 1 per row)
    relevant_urls = (
        train_df.loc[train_df[query_col] == query, relevant_col]
        .tolist()
    )

    score = recall_at_k(predicted_urls, relevant_urls, K) 
    recall_scores.append(score)

    print("\n===========================")
    print("Query:", query)
    print("Predicted (top-K):", predicted_urls[:K])
    print("Relevant URLs    :", relevant_urls)
    print(f"Recall@{K}       : {score:.3f}")

    eval_rows.append({
        "Query": query,
        "Predicted_URLs_TopK": predicted_urls[:K],
        "Relevant_URLs_All": relevant_urls,
        f"Recall@{K}": score
    })


# === Mean Recall@K across all queries ===

if recall_scores:
    mean_recall = sum(recall_scores) / len(recall_scores)
else:
    mean_recall = 0.0 

print("\n======================================")
print(f"⭐ Mean Recall@{K} = {mean_recall:.3f}")
print("======================================")

# Save per-query evaluation
eval_df = pd.DataFrame(eval_rows)
eval_df.to_excel("EEvaluation_Recall@K.xlsx", index=False)
print("Saved evaluation results to EEvaluation_Recall@K.xlsx")



Query: I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.
Predicted (top-K): ['https://www.shl.com/products/product-catalog/view/java-2-platform-enterprise-edition-1-4-fundamental/', 'https://www.shl.com/products/product-catalog/view/java-design-patterns-new/', 'https://www.shl.com/products/product-catalog/view/core-java-entry-level-new/', 'https://www.shl.com/products/product-catalog/view/java-frameworks-new/', 'https://www.shl.com/products/product-catalog/view/smart-interview-live-coding/']
Relevant URLs    : ['https://www.shl.com/solutions/products/product-catalog/view/automata-fix-new/', 'https://www.shl.com/solutions/products/product-catalog/view/core-java-entry-level-new/', 'https://www.shl.com/solutions/products/product-catalog/view/java-8-new/', 'https://www.shl.com/solutions/products/product-catalog/view/core-java-advanced-level-new/', 'https://www.shl.com/products/prod